In [46]:
# THIS IS WIP / EXPERIMENTAL

# ! pip install pandas
# ! pip install requests
# ! pip install plotly
# ! pip install datetime
# ! pip install os
# ! pip freeze = requirements.txt

In [47]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np
import os
import time
import sys

sys.path.append("../helper_functions")
import subgraph_utils as subg
import defillama_utils as dfl
import pandas_utils as pu

In [48]:
pwd = os.getcwd()
# Verify that our path is right
if "op_rewards_tracking" in pwd:
    prepend = ""
else:
    prepend = "op_rewards_tracking/"
print(pwd)
# if TVL by token is not available, do we fallback on raw TVL (sensitive to token prices)?
do_fallback_on_raw_tvl = True
str_fallback_indicator = ""  # Dont append any indicator yet, since it screws up joins

/Users/michaelsilberling/Documents/GitHub/op-analytics/op_rewards_tracking


In [49]:
# Protocol Incentive Start Dates
# Eventually, move this to its own file / csv
protocols = pd.read_csv("inputs/" + "op_incentive_protocols.csv")
# evaluate arrays as array
protocols["contracts"] = protocols["contracts"].apply(pu.str_to_list)
protocols["is_external_dex_bridge_pool"] = protocols[
    "is_external_dex_bridge_pool"
].fillna(0)
# protocols['is_external_dex_bridge_pool'] = protocols['is_external_dex_bridge_pool'].astype(str)
# display(protocols)

# If we need to map Defillama slugs to our unified App Name (Dune-Based) - Not case sensitive
protocol_name_mapping = pd.DataFrame(
    [
        ["aave-v3", "aave"],
        ["beefy", "beefy finance"],
        ["revert-compoundor", "revert finance"],
        ["cbridge", "celer"],
        ["pickle", "pickle finance"],
        ["stargate", "stargate finance"],
        ["sushi-trident", "sushi"],
    ],
    columns=["slug", "app_name"],
)

date_cols = ["start_date", "end_date"]
for d in date_cols:
    protocols[d] = pd.to_datetime(protocols[d])

protocols = protocols.merge(protocol_name_mapping, on="slug", how="left")

# For subgraphs
protocols["protocol"] = protocols["slug"]
protocols["app_name"] = (
    (protocols["app_name"].combine_first(protocols["slug"]))
    .str.replace("-", " ")
    .str.title()
)

protocols["parent_protocol"] = (
    protocols["parent_protocol"].combine_first(protocols["app_name"])
).str.capitalize()

protocols["id_format"] = protocols[
    "parent_protocol"
]  # protocols['slug'].str.replace('-',' ').str.title()
protocols["program_name"] = np.where(
    ((protocols["name"].isna())),
    protocols["slug"].str.replace("-", " ").str.title(),
    protocols["slug"].str.replace("-", " ").str.title() + " - " + protocols["name"],
)
protocols["top_level_name"] = np.where(
    protocols["name"].isna(), protocols["id_format"], protocols["name"]
)
# protocols['program_name'] = np.where( protocols['name'] == '', protocols['id_format'], protocols['name'])

protocols = protocols.sort_values(by="start_date", ascending=True)

# display(protocols)

In [50]:
# Columns about the protocol that we'll carry throughout
protocol_info_cols = [
    "protocol",
    "app_name",
    "program_name",
    "top_level_name",
    "parent_protocol",
    "include_in_summary",
    "is_external_dex_bridge_pool",
]
# protocols info
protocol_cols = ["op_source", "start_date", "end_date", "num_op", "num_op_override"]
join_cols = [
    "program_name",
    "protocol",
    "app_name",
    "top_level_name",
    "parent_protocol",
    "include_in_summary",
    "is_external_dex_bridge_pool",
]

In [51]:
# Pull Data
dfl_protocols = protocols[protocols["data_source"] == "defillama"].copy()

# drop og protocol column to avoid collisions
dfl_protocols = dfl_protocols.drop("protocol", axis=1)

dfl_slugs = dfl_protocols[["slug"]].drop_duplicates()
# display(dfl_slugs)
df_dfl = dfl.get_range(
    dfl_slugs[["slug"]], ["Optimism"], fallback_on_raw_tvl=do_fallback_on_raw_tvl
)

df_dfl["is_raw_tvl"] = np.where(df_dfl["slug"].str.endswith("*"), 1, 0)

In [52]:
# display(df_dfl[df_dfl['protocol'].str.contains('arrakis')])
# display(df_dfl)

In [53]:
# Format Columns

df_dfl = df_dfl.merge(dfl_protocols, on="slug")

df_dfl["parent_protocol"] = df_dfl["parent_protocol_y"].combine_first(
    df_dfl["parent_protocol_x"]
)

df_dfl["name"] = df_dfl["name_y"].combine_first(df_dfl["name_x"]) + np.where(
    df_dfl["protocol"].str.endswith("*"), "*", ""
)  # IF Raw TVL, pull this in
# display(df_dfl)
df_dfl["top_level_name"] = df_dfl["top_level_name"] + np.where(
    df_dfl["protocol"].str.endswith("*"), "*", ""
)  # IF Raw TVL, pull this in

df_dfl["program_name"] = df_dfl["program_name"] + np.where(
    df_dfl["protocol"].str.endswith("*"), "*", ""
)  # IF Raw TVL, pull this in

df_dfl = df_dfl[
    ["date", "token", "token_value", "usd_value", "start_date", "end_date"]
    + protocol_info_cols
]

In [54]:
subg_protocols = protocols[protocols["data_source"].str.contains("pool-")].copy()
for p in protocol_info_cols:
    subg_protocols["og_" + p] = subg_protocols[p]

subg_protocols["df_source"] = subg_protocols["data_source"].str.split("-").str[-1]

In [55]:
dfs_sub = []
for index, program in subg_protocols.iterrows():
    min_tsmp = int(pd.to_datetime(program["start_date"]).timestamp())
    min_tsmp = min_tsmp - 1000  # add some buffer
    source_slug = program["source_slug"]
    df_source = program["df_source"]
    for c in program["contracts"]:
        # messari generalized
        if df_source == "messari":
            sdf = subg.get_messari_format_pool_tvl(
                source_slug, c.lower(), min_ts=min_tsmp
            )
        # subgraph specific
        elif df_source == "curve":
            sdf = subg.get_curve_pool_tvl(c.lower(), min_ts=min_tsmp)
        elif df_source == "velodrome":
            sdf = subg.get_velodrome_pool_tvl(c.lower(), min_ts=min_tsmp)
        elif df_source == "hop":
            sdf = subg.get_hop_pool_tvl(c, min_ts=min_tsmp)

        sdf["start_date"] = program["start_date"]
        sdf["end_date"] = program["end_date"]

        for p in protocol_info_cols:
            sdf[p] = program["og_" + p]

        sdf["token_value"] = sdf["token_value"].fillna(0)
        sdf["usd_value"] = sdf["usd_value"].fillna(0)
        dfs_sub.append(sdf)
df_df_sub = pd.concat(dfs_sub)

In [56]:
df_df_comb = pd.concat([df_dfl, df_df_sub])
# remove * from protocol
df_df_comb["protocol"] = (
    df_df_comb["protocol"]
    .str[:-1]
    .where(df_df_comb["protocol"].str[-1] == "*", df_df_comb["protocol"])
)

# format dates
df_df_comb["start_date"] = pd.to_datetime(df_df_comb["start_date"])
df_df_comb["end_date"] = pd.to_datetime(df_df_comb["end_date"])
df_df_comb["date"] = pd.to_datetime(df_df_comb["date"])


# Make sure datatypes are clean
df_df_comb["token_value"] = df_df_comb["token_value"].astype("float64")
df_df_comb["usd_value"] = df_df_comb["usd_value"].astype("float64")

# create an extra day to handle for tokens dropping to 0
# this is a temp fix - longer term also: Get max of a token x date and do date + 1 = 0 (i.e. weth to eth flips)
# find intermediate gaps. Call it a 0 flow in the in-between dates (i.e. pooltogether)
df_df_shift = df_df_comb.copy()
df_df_shift["date"] = df_df_shift["date"] + timedelta(days=1)
df_df_shift["token_value"] = 0
df_df_shift["usd_value"] = 0
# merge back in
df_df = pd.concat([df_df_comb, df_df_shift])
df_df = df_df[df_df["date"] <= pd.to_datetime("today")]


# Group - Exclude End Date since this is often null and overwritting could be weird, especially if we actually know an end date
df_df["start_date"] = df_df["start_date"].fillna(pd.to_datetime("today").floor("d"))
# Generate End Date Column
df_df["end_date_30"] = df_df["end_date"].fillna(pd.to_datetime("today")).dt.floor(
    "d"
) + timedelta(days=30)

# Aggregate to handle for the work above
df_df = (
    df_df.groupby(["date", "token", "start_date", "end_date_30"] + protocol_info_cols)
    .sum()
    .reset_index()
)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/189999323.py:8: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_df_comb['date'] = pd.to_datetime(df_df_comb['date'])
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/189999323.py:8: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df_df_comb['date'] = pd.to_datetime(df_df_comb['date'])
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/189999323.py:33: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Eithe

In [57]:
data_df = df_df.copy()

# data_df = data_df[data_df['token_value'] > 0] #Exclude this, so we can read flows

data_df.sort_values(by="date", inplace=True)
# data_df['token_value'] = data_df['token_value'].replace(0, np.nan) #keep zeroes
data_df["price_usd"] = data_df["usd_value"] / data_df["token_value"]

data_df["rank_desc"] = (
    data_df.groupby(["protocol", "program_name", "token"])["date"]
    .rank(method="dense", ascending=False)
    .astype(int)
)

data_df.sort_values(by="date", inplace=True)

# Generate the last prices df
last_df = data_df[data_df["rank_desc"] == 1]
last_df = last_df.rename(columns={"price_usd": "last_price_usd"})
last_df = last_df[["token", "protocol", "program_name", "last_price_usd"]]

In [58]:
data_df = data_df.merge(last_df, on=["token", "protocol", "program_name"], how="left")

data_df["last_token_value"] = data_df.groupby(["token", "protocol", "program_name"])[
    "token_value"
].shift(1)

data_df["last_price_usd"] = data_df.groupby(["token", "protocol", "program_name"])[
    "price_usd"
].shift(1)

# If first instnace of token, make sure there's no price diff
data_df["last_price_usd"] = (
    data_df[["last_price_usd", "price_usd"]].bfill(axis=1).iloc[:, 0]
)
# Forward fill if token drops off
data_df["price_usd"] = data_df[["price_usd", "last_price_usd"]].bfill(axis=1).iloc[:, 0]

data_df["last_token_value"] = data_df["last_token_value"].fillna(0)

data_df["net_token_flow"] = data_df["token_value"] - data_df["last_token_value"]
data_df["net_price_change"] = data_df["price_usd"] - data_df["last_price_usd"]

data_df["net_dollar_flow"] = data_df["net_token_flow"] * data_df["price_usd"]
data_df["last_price_net_dollar_flow"] = (
    data_df["net_token_flow"] * data_df["last_price_usd"]
)

data_df["net_price_stock_change"] = (
    data_df["last_token_value"] * data_df["net_price_change"]
)


# display(data_df)

In [59]:
# filter before start date
data_df = data_df[data_df["date"] >= data_df["start_date"]]
# filter lte end date + 30
data_df = data_df[data_df["date"] <= data_df["end_date_30"]]
data_df.drop("end_date_30", axis=1, inplace=True)

if not os.path.exists(prepend + "csv_outputs"):
    os.mkdir(prepend + "csv_outputs")
data_df.to_csv(prepend + "csv_outputs/" + "tvl_flows_by_token.csv")

In [60]:
# ---
# Add rows for unified program views
# ---

# Remove * from mappings
data_df["top_level_name_map"] = (
    data_df["top_level_name"]
    .str[:-1]
    .where(data_df["top_level_name"].str[-1] == "*", data_df["top_level_name"])
)
data_df["program_name_map"] = (
    data_df["program_name"]
    .str[:-1]
    .where(data_df["program_name"].str[-1] == "*", data_df["program_name"])
)

tst = data_df[data_df["include_in_summary"] == 1]
tst = (
    tst.groupby(["top_level_name_map"])
    .agg({"program_name_map": "nunique"})
    .reset_index()
)
tst = tst[tst["program_name_map"] > 1]

unified_prg = data_df[
    (data_df["top_level_name_map"].isin(tst["top_level_name_map"]))
    & (data_df["include_in_summary"] == 1)
]
unified_prot = protocols[
    (protocols["name"].isin(tst["top_level_name_map"]))
    & (protocols["include_in_summary"] == 1)
][protocol_cols + join_cols + ["name"]]
unified_prg["include_in_summary"] = 0
unified_prot["include_in_summary"] = 0
# display(unified_prot)
# # set the values of the columns to the dictionary value using loc
cols_override = [
    "protocol",
    "program_name",
    "app_name",
    "top_level_name",
    "parent_protocol",
]
for i in cols_override:
    unified_prg[i] = unified_prg["top_level_name_map"]
    unified_prot[i] = unified_prot["name"]

unified_prot = unified_prot.groupby(join_cols + ["name", "op_source"]).agg(
    {"start_date": "min", "end_date": "max", "num_op": "sum", "num_op_override": "sum"}
)
unified_prot = unified_prot.reset_index()
# display(unified_prot)
unified_prot = unified_prot[protocol_cols + join_cols]

# align program info
unified_prg["start_date"] = unified_prg.groupby("program_name")["start_date"].transform(
    "min"
)
# unified_prg['end_date'] = unified_prg.groupby('program_name')['end_date'].transform('max')
# unified_prg['end_date_30'] = unified_prg.groupby('program_name')['end_date_30'].transform('max')

# APPEND BACK IN

data_df = pd.concat([data_df, unified_prg])
protocols = pd.concat(
    [protocols[protocol_cols + join_cols], unified_prot[protocol_cols + join_cols]]
)

# display(data_df)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/2112857665.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unified_prg['include_in_summary'] = 0
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/2112857665.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unified_prg[i] = unified_prg['top_level_name_map']
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/2112857665.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [61]:
display(data_df[data_df["protocol"].str.contains("clipper")])

,date,token,start_date,protocol,app_name,program_name,top_level_name,parent_protocol,include_in_summary,is_external_dex_bridge_pool,...,rank_desc,last_price_usd,last_token_value,net_token_flow,net_price_change,net_dollar_flow,last_price_net_dollar_flow,net_price_stock_change,top_level_name_map,program_name_map
128753,2023-01-23,DAI,2023-01-23,clipper,Clipper,Clipper,Clipper,Clipper,1,0.0,...,51,1.001000,87110.94246,821.34964,1.000000e-03,822.992339,822.170990,87.110941,Clipper,Clipper
128771,2023-01-23,ETH,2023-01-23,clipper,Clipper,Clipper,Clipper,Clipper,1,0.0,...,51,NaN,0.00000,0.00000,NaN,NaN,NaN,NaN,Clipper,Clipper
128834,2023-01-23,LINK,2023-01-23,clipper,Clipper,Clipper,Clipper,Clipper,1,0.0,...,51,6.900000,5186.19284,216.91444,2.400000e-01,1548.769102,1496.709636,1244.686298,Clipper,Clipper
128877,2023-01-23,OP,2023-01-23,clipper,Clipper,Clipper,Clipper,Clipper,1,0.0,...,51,2.230000,41309.96616,-1142.43979,-2.000000e-02,-2524.791936,-2547.640732,-826.199324,Clipper,Clipper
129001,2023-01-23,USDC,2023-01-23,clipper,Clipper,Clipper,Clipper,Clipper,1,0.0,...,51,1.002000,177966.90815,-407.91920,-1.000000e-03,-408.327119,-408.735038,-177.966901,Clipper,Clipper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149763,2023-03-14,OP,2023-01-23,clipper,Clipper,Clipper,Clipper,Clipper,1,0.0,...,1,2.370000,30716.15675,-1676.35999,2.800000e-01,-4442.353973,-3972.973176,8600.523883,Clipper,Clipper
149888,2023-03-14,USDC,2023-01-23,clipper,Clipper,Clipper,Clipper,Clipper,1,0.0,...,1,0.992171,140173.56540,8.43238,6.533000e-03,8.421452,8.366363,915.753908,Clipper,Clipper
149909,2023-03-14,USDT,2023-01-23,clipper,Clipper,Clipper,Clipper,Clipper,1,0.0,...,1,1.005000,62157.18487,6585.53279,9.471379e-11,6618.460454,6618.460453,0.000006,Clipper,Clipper
149929,2023-03-14,WBTC,2023-01-23,clipper,Clipper,Clipper,Clipper,Clipper,1,0.0,...,1,22556.028741,3.07647,0.00011,1.599995e+03,2.657163,2.481163,4922.335807,Clipper,Clipper


In [62]:
netdf_df = data_df[
    [
        "date",
        "net_dollar_flow",
        "net_price_stock_change",
        "last_price_net_dollar_flow",
        "usd_value",
    ]
    + protocol_info_cols
]

protocols_join = protocols.copy()

# Get net dataframe - by date (combine all tokens together)
netdf_df = netdf_df.groupby(["date"] + protocol_info_cols).sum(
    [
        "net_dollar_flow",
        "net_price_stock_change",
        "last_price_net_dollar_flow",
        "usd_value",
    ]
)

# reset & get program data
netdf_df.reset_index(inplace=True)

netdf_df["tvl_change"] = netdf_df["usd_value"] - netdf_df.groupby(
    ["protocol", "program_name", "app_name"]
)["usd_value"].shift(1)
netdf_df["error"] = netdf_df["tvl_change"] - (
    netdf_df["net_dollar_flow"] + netdf_df["net_price_stock_change"]
)

cumul_cols = ["net_dollar_flow", "last_price_net_dollar_flow", "net_price_stock_change"]
for c in cumul_cols:
    netdf_df["cumul_" + c] = netdf_df.groupby(["protocol", "program_name"])[c].cumsum()

# Bring Program info Back In

# display(netdf_df)


join_cols_join = [col + "_join" for col in join_cols]
for c in join_cols:
    # Convert to strings - handle for weird datatype issues
    netdf_df[c + "_join"] = netdf_df[c].astype(str)
    protocols_join[c + "_join"] = protocols_join[c].astype(str)

    # rm star
    netdf_df[c + "_join"] = (
        netdf_df[c + "_join"]
        .str[:-1]
        .where(netdf_df[c].astype(str).str[-1] == "*", netdf_df[c])
        .astype(str)
    )

    # force lower
    netdf_df[c + "_join"] = netdf_df[c + "_join"].str.strip().str.lower()
    protocols_join[c + "_join"] = protocols_join[c + "_join"].str.strip().str.lower()


all_protocol_cols = protocol_cols + join_cols_join
# print(all_protocol_cols)
display(netdf_df[join_cols_join].dtypes)
display(protocols_join[join_cols_join].dtypes)

netdf_df = netdf_df.merge(
    protocols_join[all_protocol_cols], on=join_cols_join, how="left"
)

# For Summary
if_ended_cols = ["net_dollar_flow", "last_price_net_dollar_flow"]
new_ended_cols = []
for e in if_ended_cols:
    netdf_df["cumul_" + e + "_if_ended"] = (
        netdf_df[~netdf_df["end_date"].isna()]
        .groupby(["protocol", "program_name"])[e]
        .cumsum()
    )
    new_ended_cols.append("cumul_" + e + "_if_ended")


for d in date_cols:
    netdf_df[d] = pd.to_datetime(netdf_df[d])

# check info at program end
# display(program_end_df)
display(netdf_df[netdf_df["protocol"] == "perpetual-protocol"])

program_name_join                   object
protocol_join                       object
app_name_join                       object
top_level_name_join                 object
parent_protocol_join                object
include_in_summary_join             object
is_external_dex_bridge_pool_join    object
dtype: object

program_name_join                   object
protocol_join                       object
app_name_join                       object
top_level_name_join                 object
parent_protocol_join                object
include_in_summary_join             object
is_external_dex_bridge_pool_join    object
dtype: object

,date,protocol,app_name,program_name,top_level_name,parent_protocol,include_in_summary,is_external_dex_bridge_pool,net_dollar_flow,net_price_stock_change,...,parent_protocol_join,include_in_summary_join,is_external_dex_bridge_pool_join,op_source,start_date,end_date,num_op,num_op_override,cumul_net_dollar_flow_if_ended,cumul_last_price_net_dollar_flow_if_ended
1,2022-07-14,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,-118571.054762,0.000002,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
3,2022-07-15,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,85964.761152,0.000008,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
7,2022-07-16,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,9454.523080,-27897.413700,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
11,2022-07-17,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,63494.336352,-7590.668162,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
15,2022-07-18,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,6392.603898,35578.323152,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7258,2023-03-10,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,-10214.419683,34912.526203,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
7304,2023-03-11,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,206405.076164,-165912.116844,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
7347,2023-03-12,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,-558625.989217,-535288.024833,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
7390,2023-03-13,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,133845.992337,512237.517143,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN


In [63]:
# display(netdf_df[netdf_df['protocol'].str.contains('velodr')])
display(netdf_df)

,date,protocol,app_name,program_name,top_level_name,parent_protocol,include_in_summary,is_external_dex_bridge_pool,net_dollar_flow,net_price_stock_change,...,parent_protocol_join,include_in_summary_join,is_external_dex_bridge_pool_join,op_source,start_date,end_date,num_op,num_op_override,cumul_net_dollar_flow_if_ended,cumul_last_price_net_dollar_flow_if_ended
0,2022-07-13,velodrome,Velodrome,Velodrome - Velodrome #1 *,Velodrome #1 *,Velodrome,1,0.0,1.286083e+05,0.000000e+00,...,velodrome,1,0.0,Partner Fund,2022-07-13,2022-11-17,3000000.0,3000000.0,1.286083e+05,1.286083e+05
1,2022-07-14,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,-1.185711e+05,2.151991e-06,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
2,2022-07-14,velodrome,Velodrome,Velodrome - Velodrome #1 *,Velodrome #1 *,Velodrome,1,0.0,1.736532e+06,0.000000e+00,...,velodrome,1,0.0,Partner Fund,2022-07-13,2022-11-17,3000000.0,3000000.0,1.865141e+06,1.865141e+06
3,2022-07-15,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,8.596476e+04,7.713076e-06,...,perpetual protocol,1,0.0,Governance - Phase 0,2022-07-14,NaT,9000000.0,NaN,NaN,NaN
4,2022-07-15,rubicon,Rubicon,Rubicon,Rubicon,Rubicon,1,0.0,2.904547e+03,1.007550e+04,...,rubicon,1,0.0,Governance - Phase 0,2022-07-15,2023-02-10,900000.0,NaN,2.904547e+03,2.676995e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7450,2023-03-14,synthetix,Synthetix,Synthetix - sETH-ETH: Curve,sETH-ETH: Curve,Synthetix,1,1.0,1.542521e+05,1.500778e+06,...,synthetix,1,1.0,Governance - Phase 0,2022-08-25,NaT,NaN,NaN,NaN,NaN
7451,2023-03-14,synthetix,Synthetix,Synthetix - sUSD-3Crv: Curve,sUSD-3Crv: Curve,Synthetix,1,1.0,3.341992e+06,-2.094482e+04,...,synthetix,1,1.0,Governance - Phase 0,2022-08-25,NaT,NaN,NaN,NaN,NaN
7452,2023-03-14,thales,Thales,Thales,Thales,Thales,1,0.0,1.329747e+04,-4.006353e+01,...,thales,1,0.0,Governance - Phase 0,2022-07-15,NaT,900000.0,NaN,NaN,NaN
7453,2023-03-14,velodrome,Velodrome,Velodrome - Velodrome #2 (Tour de OP)*,Velodrome #2 (Tour de OP)*,Velodrome,1,0.0,2.070332e+07,0.000000e+00,...,velodrome,1,0.0,Governance - Season 2,2022-11-24,NaT,4000000.0,NaN,NaN,NaN


In [64]:
summary_cols = [
    "cumul_net_dollar_flow",
    "cumul_last_price_net_dollar_flow",
    "cumul_net_price_stock_change",
    "num_op_override",
]

netdf_df["program_rank_desc"] = (
    netdf_df.groupby(["protocol", "program_name"])["date"]
    .rank(method="dense", ascending=False)
    .astype(int)
)

# for sc in summary_cols:
#         netdf_df[sc] = netdf_df[sc].astype('int64')
summary_cols = summary_cols + new_ended_cols
# print(summary_cols)
program_end_df = (
    netdf_df[
        (
            pd.to_datetime(netdf_df["date"]) == pd.to_datetime(netdf_df["end_date"])
        )  # is at end date
        | (netdf_df["program_rank_desc"] == 1)  # or is latest date
    ]
    .groupby(join_cols)
    .sum(numeric_only=True)
)
program_end_df.reset_index(inplace=True)
# display(program_end_df)

# display(program_end_df)
for s in summary_cols:
    s_new = s + "_at_program_end"
    program_end_df = program_end_df.rename(columns={s: s_new})
    netdf_df = netdf_df.merge(
        program_end_df[["protocol", "program_name", s_new]],
        on=["protocol", "program_name"],
        how="left",
    )
    netdf_df = netdf_df.merge(
        program_end_df[protocol_info_cols + [s_new]], on=protocol_info_cols, how="left"
    )

# netdf_df['cumul_net_dollar_flow_at_program_end'] = netdf_df[is_program_end].groupby(['protocol', 'program_name']).sum(['cumul_net_dollar_flow'])
# netdf_df['cumul_last_price_net_dollar_flow_at_program_end'] = netdf_df[netdf_df['date'] == netdf_df['end_date']]['last_price_net_dollar_flow'].groupby(['protocol', 'program_name']).cumsum()
# netdf_df['cumul_net_price_stock_change_at_program_end'] = netdf_df[netdf_df['date'] == netdf_df['end_date']]['net_price_stock_change'].groupby(['protocol', 'program_name']).cumsum()

# netdf_df.loc[ netdf_df['end_date'] == pd.to_datetime("2000-01-01"), 'end_date' ] == pd.to_datetime("1900-01-01")

# np.where( netdf_df['end_date'] <= pd.to_datetime("2000-01-01") , pd.NaT , netdf_df['end_date'] )
# display(netdf_df[netdf_df['protocol'] == 'hundred-finance'].sort_values(by='program_rank_desc'))

In [65]:
# netdf_df[(netdf_df['date'] >= '2022-10-06') & (netdf_df['date'] <= '2022-10-12')].tail(10)
# netdf_df[netdf_df['protocol'].str.contains('velodr')]

In [66]:
during_str = "During Program"
post_str = "Post-Program"

netdf_df["period"] = np.where(
    netdf_df["date"] > netdf_df["end_date"], post_str, during_str
)
if not os.path.exists(prepend + "csv_outputs"):
    os.mkdir(prepend + "csv_outputs")
netdf_df.to_csv(prepend + "csv_outputs/op_summer_daily_stats.csv", index=False)

# SORT FOR CHARTS
netdf_df = netdf_df.sort_values(
    by=["top_level_name", "program_name", "app_name", "parent_protocol"],
    ascending=[True, True, True, True],
)
# display(netdf_df.head())
# print(netdf_df.columns)

In [67]:
latest_data_df = netdf_df[netdf_df["program_rank_desc"] == 1]
latest_data_df["date"] = latest_data_df["date"].dt.date
# latest_data_df['days_since_program_end']
# latest_data_df.loc[latest_data_df['end_date'] != '', 'days_since_program_end'] = \
#         pd.to_datetime(latest_data_df['end_date']) \
#         - pd.to_datetime(latest_data_df['date'])

latest_data_df["days_since_program_end"] = np.where(
    latest_data_df["end_date"] != "",
    pd.to_datetime(latest_data_df["end_date"]) - pd.to_datetime(latest_data_df["date"]),
    pd.to_datetime(latest_data_df["date"])
    - pd.to_datetime(latest_data_df["start_date"]),
)
latest_data_df = latest_data_df.sort_values(by="start_date", ascending=False)
display(latest_data_df)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/973706550.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_data_df['date'] = latest_data_df['date'].dt.date
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/973706550.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_data_df['days_since_program_end'] = \


,date,protocol,app_name,program_name,top_level_name,parent_protocol,include_in_summary,is_external_dex_bridge_pool,net_dollar_flow,net_price_stock_change,...,cumul_net_price_stock_change_at_program_end_x,cumul_net_price_stock_change_at_program_end_y,num_op_override_at_program_end_x,num_op_override_at_program_end_y,cumul_net_dollar_flow_if_ended_at_program_end_x,cumul_net_dollar_flow_if_ended_at_program_end_y,cumul_last_price_net_dollar_flow_if_ended_at_program_end_x,cumul_last_price_net_dollar_flow_if_ended_at_program_end_y,period,days_since_program_end
7443,2023-03-14,sushi-trident,Sushi,Sushi Trident,Sushi,Sushi,1,0.0,9.630700e+05,22198.898399,...,-1.804873e+05,-1.804873e+05,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,During Program,NaT
7416,2023-03-14,angle,Angle,Angle - agEUR/MAI: Velodrome,agEUR/MAI: Velodrome,Angle,1,1.0,-1.451048e+05,-54756.459574,...,-2.422895e+05,-2.422895e+05,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,During Program,NaT
7414,2023-03-14,angle,Angle,Angle - OP/agEUR: Velodrome,OP/agEUR: Velodrome,Angle,1,1.0,6.686973e+04,-8087.007763,...,-9.573163e+04,-9.573163e+04,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,During Program,NaT
7415,2023-03-14,angle,Angle,Angle - USDC/agEUR: Uniswap,USDC/agEUR: Uniswap,Angle,1,1.0,-2.965538e+02,0.000000,...,1.109634e+04,1.109634e+04,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,During Program,NaT
7441,2023-03-14,rocket-pool,Rocket Pool,Rocket Pool - vWETH/rETH: Velodrome,vWETH/rETH: Velodrome,Rocket pool,1,1.0,1.765388e+06,389389.178441,...,9.832848e+05,9.832848e+05,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,During Program,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,2022-09-21,layer2dao,Layer2Dao,Layer2Dao - L2DAO/OP: Velodrome,L2DAO/OP: Velodrome,Layer2dao,1,1.0,-1.378956e+03,-6277.440473,...,-2.600964e+04,-2.600964e+04,0.0,0.0,4.681078e+05,4.681078e+05,3.585146e+05,3.585146e+05,Post-Program,-30 days
7357,2023-03-12,rubicon,Rubicon,Rubicon,Rubicon,Rubicon,1,0.0,-1.479401e+04,-88950.334782,...,3.839408e+06,3.839408e+06,0.0,0.0,9.296232e+06,9.296232e+06,9.090737e+06,9.090737e+06,Post-Program,-30 days
7452,2023-03-14,thales,Thales,Thales,Thales,Thales,1,0.0,1.329747e+04,-40.063531,...,-3.928285e+03,-3.928285e+03,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,During Program,NaT
7432,2023-03-14,perpetual-protocol,Perpetual Protocol,Perpetual Protocol,Perpetual protocol,Perpetual protocol,1,0.0,2.524486e+05,112422.136289,...,-4.094812e+04,-4.094812e+04,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,During Program,NaT


In [68]:
# Generate agg summary df
season_summary_pds = latest_data_df[latest_data_df["include_in_summary"] == 1].copy()

season_summary_s0_no_perp = season_summary_pds[
    (season_summary_pds["op_source"] == "Gov Fund - Phase 0")
    & (season_summary_pds["protocol"] != "perpetual-protocol")
]

season_summary_s0_no_perp["op_source"] = "Gov Fund - Phase 0 (Excl. Perp)"

season_summary_raw = pd.concat([season_summary_pds, season_summary_s0_no_perp])

season_summary_completed_raw = season_summary_pds[
    season_summary_pds["end_date"] < pd.to_datetime("today")
]  # only ended summaries

In [69]:
# SEASON SUMMARY
season_summary = season_summary_raw.groupby("op_source").sum()
# display(season_summary.head())
season_summary.reset_index()
# create a row with total values
season_summary_total_raw = season_summary_raw.copy()
season_summary_total_raw["op_source"] = "- TOTAL -"
season_summary_total = pd.DataFrame(season_summary_total_raw.groupby("op_source").sum())

# concatenate the aggregated grouped data with the total row
season_summary = pd.concat([season_summary, season_summary_total])
season_summary.reset_index(inplace=True)
# season_summary.head()

# SEASON SUMMARY IF COMPLETED - loops were weird, so doing it this way

season_summary_completed = season_summary_completed_raw.groupby("op_source").sum()
# display(season_summary.head())
season_summary_completed.reset_index()
# create a row with total values
season_summary_completed_total_raw = season_summary_completed_raw.copy()
season_summary_completed_total_raw["op_source"] = "- TOTAL -"
season_summary_completed_total = pd.DataFrame(
    season_summary_completed_total_raw.groupby("op_source").sum()
)

# concatenate the aggregated grouped data with the total row
season_summary_completed = pd.concat(
    [season_summary_completed, season_summary_completed_total]
)
season_summary_completed.reset_index(inplace=True)
# season_summary.head()

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/2825843958.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  season_summary = season_summary_raw.groupby('op_source').sum()
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/2825843958.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  season_summary_total = pd.DataFrame(season_summary_total_raw.groupby('op_source').sum())
/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88030/2825843958.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only wil

In [70]:
# print(latest_data_df.columns)
# print(season_summary.columns)

In [71]:
df_list = [latest_data_df, season_summary, season_summary_completed]
latest_data_df.name = "op_summer_latest"
season_summary.name = "season_summary"
season_summary_completed.name = "season_summary_completed"

for df in df_list:
    # Fix 0 columns
    for col in df.columns:
        if "_at_program_end" in col:
            df[col] = df[col].astype(float)
            df[col] = np.where(df[col] == 0, np.NaN, df[col])

    # Per OP Metrics Migrated to combined deployment measures
    # df['cumul_flows_per_op_at_program_end'] = df['cumul_net_dollar_flow_at_program_end'] / df['num_op_at_program_end']

    # df['cumul_flows_per_op_latest'] = df['cumul_net_dollar_flow'] / df['num_op']

    # df['last_price_net_dollar_flows_per_op_at_program_end'] = df['cumul_last_price_net_dollar_flow_at_program_end'] / df['num_op_at_program_end']
    # df['last_price_net_dollar_flows_per_op_latest'] = df['cumul_last_price_net_dollar_flow'] / df['num_op']

    df["flows_retention"] = (
        df["cumul_net_dollar_flow_if_ended"]
        / df["cumul_net_dollar_flow_at_program_end"]
        * np.where(df["cumul_net_dollar_flow"] < 0, -1, 1)
    )
    df["last_price_net_dollar_flows_retention"] = (
        df["cumul_last_price_net_dollar_flow_if_ended"]
        / df["cumul_last_price_net_dollar_flow_at_program_end"]
        * np.where(df["cumul_last_price_net_dollar_flow"] < 0, -1, 1)
    )

KeyError: 'cumul_net_dollar_flow_at_program_end'

In [ ]:
for df in df_list:
    # display(df)
    # get df name
    col_list = [
        "date",
        "include_in_summary",
        "top_level_name",
        "parent_protocol",
        "program_name",
        "app_name",
        "num_op",
        "num_op_override",
        "period",
        "op_source",
        "start_date",
        "end_date",
        "cumul_net_dollar_flow_at_program_end",
        "cumul_net_dollar_flow"
        # ,'cumul_flows_per_op_at_program_end'
        ,
        "cumul_last_price_net_dollar_flow_at_program_end"
        # ,'cumul_flows_per_op_latest'
        ,
        "cumul_last_price_net_dollar_flow"
        # , 'last_price_net_dollar_flows_per_op_at_program_end'
        # ,'last_price_net_dollar_flows_per_op_latest'
        ,
        "flows_retention",
        "last_price_net_dollar_flows_retention",
    ]
    summary_exclude_list = [
        "date",
        "top_level_name",
        "program_name",
        "app_name",
        "period",
        "start_date",
        "end_date",
        "parent_protocol",
    ]
    sort_cols = ["Start", "# OP"]

    if df.name == "op_summer_latest":
        html_name = df.name + "_stats"
        sort_order = [False, False]
    elif "season_summary" in df.name:
        html_name = df.name + "_stats"
        sort_cols = ["Source", "# OP"]
        sort_order = [False, True]  # so totals goes to bottom
        col_list = [x for x in col_list if x not in summary_exclude_list]
    else:
        html_name = "other"

    df_format = df.copy()
    new_cols = df_format.columns
    drop_cols = [
        "net_dollar_flow",
        "net_price_stock_change",
        "last_price_net_dollar_flow",
        "usd_value",
        "tvl_change",
        "error",
    ]
    new_cols = new_cols.drop(drop_cols)
    # print(new_cols)
    df_format = df_format[new_cols]

    # df_format['num_op'] = df_format['num_op'].apply(lambda x: '{0:,.0f}'.format(x) if not pd.isna(x) else x )
    # df_format['flows_retention'] = df_format['flows_retention'].apply(lambda x: '{:,.1%}'.format(x) if not pd.isna(x) else x )
    # df_format['last_price_net_dollar_flows_retention'] = df_format['last_price_net_dollar_flows_retention'].apply(lambda x: '{:,.1%}'.format(x) if not pd.isna(x) else x )

    df_format = df_format[col_list]
    df_format = df_format.reset_index(drop=True)

    if not os.path.exists(prepend + "csv_outputs"):
        os.mkdir(prepend + "csv_outputs")
    if not os.path.exists(prepend + "img_outputs"):
        os.mkdir(prepend + "img_outputs")
        os.mkdir(prepend + "img_outputs/overall")
        os.mkdir(prepend + "img_outputs/overall/png")
        os.mkdir(prepend + "img_outputs/overall/svg")
        os.mkdir(prepend + "img_outputs/overall/html")
    df_format.to_csv(prepend + "csv_outputs/" + html_name + ".csv", index=False)

    format_cols = [
        "cumul_flows_per_op_at_program_end",
        "cumul_flows_per_op_latest",
        "last_price_net_dollar_flows_per_op_at_program_end",
        "last_price_net_dollar_flows_per_op_latest",
    ]
    format_mil_cols = [
        "cumul_net_dollar_flow",
        "cumul_last_price_net_dollar_flow",
        "cumul_net_dollar_flow_at_program_end",
        "cumul_last_price_net_dollar_flow_at_program_end",
    ]
    # for f in format_cols:
    # df_format[f] = df_format[f].apply(lambda x: '${0:,.2f}'.format(x) if not pd.isna(x) else x )
    # df_format[f] = df_format[f].apply(lambda x: round(x,1) if not pd.isna(x) else x )
    # for fm in format_mil_cols:
    #     df_format[fm] = df_format[fm].apply(lambda x: '${0:,.2f}M'.format(x/1e6) if not pd.isna(x) else x )

    df_format = df_format.rename(
        columns={
            "date": "Date",
            "program_name": "Program",
            "num_op": "# OP",
            "period": "Period",
            "op_source": "Source",
            "start_date": "Start",
            "end_date": "End",
            "cumul_net_dollar_flow_at_program_end": "Net Flows (at End Date)",
            "cumul_net_dollar_flow": "Net Flows (End + 30)",
            "cumul_flows_per_op_at_program_end": "Net Flows per OP (at End Date)",
            "cumul_flows_per_op_latest": "Net Flows per OP (End + 30)"
            ##
            ,
            "cumul_last_price_net_dollar_flow_at_program_end": "Net Flows @ Current Prices (at End Date)",
            "cumul_last_price_net_dollar_flow": "Net Flows @ Current Prices (End + 30)",
            "last_price_net_dollar_flows_per_op_at_program_end": "Net Flows per OP @ Current Prices (at End Date)",
            "last_price_net_dollar_flows_per_op_latest": "Net Flows per OP @ Current Prices (End + 30)",
            "flows_retention": "Net Flows Retained",
            "last_price_net_dollar_flows_retention": "Net Flows Retained @ Current Prices",
        }
    )

    df_col_list = list(df_format.columns)
    df_col_list.remove("include_in_summary")

    format_mil_cols_clean = [
        x for x in df_col_list if ("Flows" in x) & ("Retained" not in x)
    ]
    # print(format_mil_cols_clean)
    format_pct_cols_clean = [x for x in df_col_list if "Retained" in x]

    format_op_cols_clean = ["# OP"]
    # [
    #     '# OP','Net Flows (at End Date)',
    #     'Net Flows (End + 30)', 'Net Flows @ Current Prices (End + 30)',
    #     'Net Flows @ Current Prices (at End Date)',
    #     'Net Flows @ Current Prices (at End Date)'
    # ]
    df_format = df_format.fillna("")
    df_format = df_format.reset_index(drop=True)
    df_format = df_format.sort_values(by=sort_cols, ascending=sort_order)

    # df_format.to_html(
    #     prepend + "img_outputs/app/" + html_name + ".html",
    #     classes='table table-stripped')
    # display(df_format[format_mil_cols_clean])
    # fig_tbl = px.table(df_format[df_col_list], sortable=True)
    # fig_tbl.show()

    # chatgpt goat?
    header = dict(
        values=df_col_list, fill_color="darkgray", align="center"
    )  # , sort_action='native')

    # format the numbers in mil_columns and store the result in a list of lists
    values = [
        [
            pu.format_num(x, "$")
            if col in format_mil_cols_clean
            else pu.format_num(x)
            if col in format_op_cols_clean
            else pu.format_pct(x)
            if col in format_pct_cols_clean
            else x
            for x in df_format[col]
        ]
        for col in df_col_list
    ]

    cells = dict(
        values=values,
        fill_color=["white", "lightgray"] * (len(df_format) // 2 + 1),
        align="right",
    )  # , line_break=True)

    data = [go.Table(header=header, cells=cells)]

    layout = go.Layout(title="TVL & Flows Stats")  # , width='100%')

    fig_tbl = go.Figure(data=data, layout=layout)
    # fig_tbl.show()
    # pd_html = pu.generate_html(df_format[df_col_list])
    # pd_html = pu.DataTable(df_format[df_col_list]).data

    # print(type(pd_html))
    # open(prepend + "img_outputs/app/html/" + html_name + ".html", "w").write(pd_html)

    if not os.path.exists(prepend + "img_outputs/app"):
        os.mkdir(prepend + "img_outputs/app")
        os.mkdir(prepend + "img_outputs/app/png")
        os.mkdir(prepend + "img_outputs/app/svg")
        os.mkdir(prepend + "img_outputs/app/html")

    fig_tbl.write_html(
        prepend + "img_outputs/app/html/" + html_name + ".html", include_plotlyjs="cdn"
    )

In [ ]:
# Filter for Charts

netdf_df = netdf_df[netdf_df["date"] <= pd.to_datetime("today").floor("d")]
# netdf_df = netdf_df[netdf_df['include_in_summary'] == 1]

In [ ]:
fig = px.line(
    netdf_df,
    x="date",
    y="net_dollar_flow",
    color="program_name",
    title="Daily Liquidity Flows Since Program Announcement",
    labels={"date": "Day", "net_dollar_flow": "Net Liquidity Flows (USD)"},
)
fig.update_layout(legend_title="App Name")
fig.update_layout(yaxis_tickprefix="$")
fig.write_image(prepend + "img_outputs/overall/svg/daily_ndf.svg")
fig.write_image(prepend + "img_outputs/overall/png/daily_ndf.png")
fig.write_html(prepend + "img_outputs/overall/daily_ndf.html", include_plotlyjs="cdn")


cumul_fig = go.Figure()
proto_names = netdf_df["program_name"].drop_duplicates()
# print(proto_names)
for p in proto_names:
    cumul_fig.add_trace(
        go.Scatter(
            x=netdf_df[netdf_df["program_name"] == p]["date"],
            y=netdf_df[netdf_df["program_name"] == p]["cumul_net_dollar_flow"],
            name=p,
            fill="tozeroy",
        )
    )  # fill down to xaxis

cumul_fig.update_layout(yaxis_tickprefix="$")
cumul_fig.update_layout(
    title="Cumulative Net Liquidity Flows Since Program Announcement<br><sup>For Ended Programs, we show continue to show flows through 30 days after program end. | * Shows raw TVL change, rather than flows</sup>",
    xaxis_title="Day",
    yaxis_title="Cumulative Net Liquidity Flows (USD)",
    legend_title="App Name",
    #     color_discrete_map=px.colors.qualitative.G10
)
cumul_fig.write_image(prepend + "img_outputs/overall/svg/cumul_ndf.svg")  # prepend +
cumul_fig.write_image(prepend + "img_outputs/overall/png/cumul_ndf.png")  # prepend +
cumul_fig.write_html(
    prepend + "img_outputs/overall/cumul_ndf.html", include_plotlyjs="cdn"
)


fig_last = go.Figure()
proto_names = netdf_df["program_name"].drop_duplicates()
# print(proto_names)
for p in proto_names:
    fig_last.add_trace(
        go.Scatter(
            x=netdf_df[netdf_df["program_name"] == p]["date"],
            y=netdf_df[netdf_df["program_name"] == p][
                "cumul_last_price_net_dollar_flow"
            ],
            name=p,
            fill="tozeroy",
        )
    )  # fill down to xaxis

fig_last.update_layout(yaxis_tickprefix="$")
fig_last.update_layout(
    title="Cumulative Net Flows since Program Announcement (At Most Recent Token Price)<br><sup>For Ended Programs, we show continue to show flows through 30 days after program end. | * Shows raw TVL change, rather than flows</sup>",
    xaxis_title="Day",
    yaxis_title="Cumulative Net Flows (USD) - At Most Recent Price",
    legend_title="App Name",
    #     color_discrete_map=px.colors.qualitative.G10
)
fig_last.write_image(prepend + "img_outputs/overall/svg/cumul_ndf_last_price.svg")
fig_last.write_image(prepend + "img_outputs/overall/png/cumul_ndf_last_price.png")
fig_last.write_html(
    prepend + "img_outputs/overall/cumul_ndf_last_price.html", include_plotlyjs="cdn"
)
# cumul_fig.show()

In [ ]:
# Program-Specific Charts

value_list = ["cumul_net_dollar_flow", "cumul_last_price_net_dollar_flow"]

for val in value_list:
    if val == "cumul_last_price_net_dollar_flow":
        postpend = " - At Last Price"
        folder_path = "/last_price"
    else:
        postpend = ""
        folder_path = ""
    proto_names = netdf_df["program_name"].drop_duplicates()
    # print(proto_names)
    for p in proto_names:
        cumul_fig_app = go.Figure()
        p_df = netdf_df[netdf_df["program_name"] == p]
        # cumul_fig_app = px.area(p_df, x="date", y="cumul_net_dollar_flow", color="period")

        during_df = p_df[p_df["period"] == during_str]
        cumul_fig_app.add_trace(
            go.Scatter(
                x=during_df["date"], y=during_df[val], name=during_str, fill="tozeroy"
            )
        )  # fill down to xaxis

        post_df = p_df[p_df["period"] == post_str]
        cumul_fig_app.add_trace(
            go.Scatter(x=post_df["date"], y=post_df[val], name=post_str, fill="tozeroy")
        )  # fill down to xaxis

        cumul_fig_app.update_layout(yaxis_tickprefix="$")
        cumul_fig_app.update_layout(
            title=p
            + "<br><sup>Cumulative Net Flows since Program Announcement, Until Program End + 30 Days"
            + postpend
            + "</sup>",
            xaxis_title="Day",
            yaxis_title="Cumulative Net Flows (USD)",
            legend_title="Period",
            #     color_discrete_map=px.colors.qualitative.G10
        )

        if not os.path.exists(prepend + "img_outputs/app" + folder_path):
            os.mkdir(prepend + "img_outputs/app" + folder_path)
        if not os.path.exists(prepend + "img_outputs/app" + folder_path + "/svg"):
            os.mkdir(prepend + "img_outputs/app" + folder_path + "/svg")
        if not os.path.exists(prepend + "img_outputs/app" + folder_path + "/png"):
            os.mkdir(prepend + "img_outputs/app/" + folder_path + "/png")

        p_file = p
        p_file = p_file.replace(" ", "_")
        p_file = p_file.replace(":", "")
        p_file = p_file.replace("/", "-")
        cumul_fig_app.write_image(
            prepend
            + "img_outputs/app"
            + folder_path
            + "/svg/cumul_ndf_"
            + p_file
            + ".svg"
        )  # prepend +
        cumul_fig_app.write_image(
            prepend
            + "img_outputs/app"
            + folder_path
            + "/png/cumul_ndf_"
            + p_file
            + ".png"
        )  # prepend +
        cumul_fig_app.write_html(
            prepend
            + "img_outputs/app"
            + folder_path
            + "/cumul_ndf_"
            + p_file
            + ".html",
            include_plotlyjs="cdn",
        )
        # cumul_fig_app.show()

In [ ]:
fig_last.show()
print("yay")

In [ ]:
#

In [ ]:
# ! jupyter nbconvert --to python optimism_incentives_app_net_flows.ipynb